In [20]:
!pip install Counter

In [28]:
from collections import Counter


with open(r"C:\Users\muham\Desktop\rüya\ruyaislenmisveriyeni.txt", 'r', encoding='utf-8') as dosya:
    metin = dosya.read() 


kelimeler = metin.split() 


kelime_sayilari = Counter(kelimeler)


en_fazla_gecen_25 = kelime_sayilari.most_common(25)

print(en_fazla_gecen_25)

[('bir', 8516), ('olarak', 2769), ('olan', 1680), ('s', 908), ('rüya', 820), ('olduğu', 754), ('kendi', 692), ('ancak', 680), ('sonra', 660), ('kadar', 657), ('zaman', 652), ('göre', 633), ('yer', 621), ('ortaya', 619), ('yeni', 606), ('içinde', 581), ('olduğunu', 565), ('aynı', 524), ('onun', 514), ('önemli', 511), ('ilk', 505), ('oyunculuk', 500), ('tarafından', 495), ('iki', 467), ('büyük', 453)]


In [29]:
from gensim.models import Word2Vec
import os


dosya_yolu = r"C:\Users\muham\Desktop\rüya\ruyaislenmisveriyeni.txt"


if os.path.exists(dosya_yolu):
    with open(dosya_yolu, 'r', encoding='utf-8') as dosya:
        metin = dosya.read().lower()  
        
    
    kelime_listesi = metin.split()  
    
   
    model = Word2Vec(sentences=[kelime_listesi], vector_size=100, window=5, min_count=1, workers=4)
    
  
    model.save("egitilmis_word2vec_model")
    
    print("Word2Vec modeli başarıyla eğitildi ve kaydedildi.")
else:
    print("Dosya bulunamadı.")

Word2Vec modeli başarıyla eğitildi ve kaydedildi.


In [30]:
from gensim.models import Word2Vec


model = Word2Vec.load("egitilmis_word2vec_model") 


kelimeler = ["olan", "bir", "olarak","rüya","olduğu","kendi","ancak","sonra","kadar","zaman","göre","yer","ortaya","yeni","içinde","olduğunu","aynı","onun","önemli","ilk"]  # Analiz yapmak istediğiniz kelimeler
benzer_kelimeler = {}

for kelime in kelimeler:
    benzer_kelimeler[kelime] = [kel for kel, _ in model.wv.most_similar(kelime, topn=5)]


for kelime, benzerler in benzer_kelimeler.items():
    print(f"{kelime}: {benzerler}")

olan: ['26864', '31287', '32014', 'süreli', '18109']
bir: ['hunlarda', 'kaçtı', 'devletletirilmesiyle', '12937', 'tembellik']
olarak: ['sırtında', 'rüyametinden', 'gidiatına', 'koyar', '12255']
rüya: ['8999', 'planlama', 'zebrayı', 'çalmış', 'körüklediği']
olduğu: ['aristandrosun', '15641', '6899', '30579', 'kir']
kendi: ['ışıklara', 'yanıtlanabilir', 'diyalogların', '24796', 'robertson']
ancak: ['kapısının', '11808', 'kalanı', '28128', '28587']
sonra: ['birleştirilmiş', '14250', 'kabustan', '21001', 'klariosa']
kadar: ['neresi', 'bulunmasına', '1126', 'istekler', '25342']
zaman: ['adalete', 'abartıya', '10100', '22789', '5725']
göre: ['yayılış', 'grebeneli', '31727', '23095', 'bendaga']
yer: ['ilemlerini', 'göstermekten', '13279', '14465', '26611']
ortaya: ['15766', 'gerekliliklerini', 'diyalogları', 'donatılmış', 'önbayrak']
yeni: ['fleming', 'airlerinin', 'röportajı', 'hanioğlunun', 'he']
içinde: ['tutma', 'hatırlarlar', 'gelirler', '943', '1328']
olduğunu: ['12802', 'görebilmekteyi

In [31]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize


dosya_yolu = r"C:\Users\muham\Desktop\rüya\ruyaislenmisveriyeni.txt"


if os.path.exists(dosya_yolu):
    with open(dosya_yolu, 'r', encoding='utf-8') as dosya:
        metin = dosya.readlines()  
    
  
    cumle_listesi = [cumle.strip() for cumle in metin if cumle.strip()] 

   
    token_cumleler = [word_tokenize(cumle.lower()) for cumle in cumle_listesi]

  
    tagged_cumleler = [TaggedDocument(words=cumle, tags=[str(i)]) for i, cumle in enumerate(token_cumleler)]

   
    model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
    model.build_vocab(tagged_cumleler)
    model.train(tagged_cumleler, total_examples=model.corpus_count, epochs=model.epochs)

  
    model.save("egitilmis_doc2vec_model")

    print("Doc2Vec modeli başarıyla eğitildi ve kaydedildi.")
else:
    print("Dosya bulunamadı.")

Doc2Vec modeli başarıyla eğitildi ve kaydedildi.


In [32]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize


model = Doc2Vec.load("egitilmis_doc2vec_model")


#6 TANE CÜMLE OLUŞTURMAMIN SEBEBİ AYNI CÜMLEYİDE BENZER CÜMLE DİYE OLUŞTURMASIDIR


def benzer_cumleleri_bul(model, cümle, n=6):
    token_cümle = word_tokenize(cümle.lower())
    cümle_vektoru = model.infer_vector(token_cümle)
    
    benzer_cümleler = model.docvecs.most_similar([cümle_vektoru], topn=n)
    
    return [benzerlik[0] for benzerlik in benzer_cümleler]


örnek_cümleler = [
    "rüyaların bilimselliği söz konusu olmadan önce çeşitli varsayımlar rüyaların nedenlerine egemen olmuştur",
    "rüyalar insanlığın başlangıcından itibaren ilgi çekici bir merak konusu olmuştur",
    "rüya aleminin psikolojinin çalışmalarında yön belirleyici olması doğanın öğrenim döneminin başlarından itibaren kendini göstermiştir",
    "rüyalarının bireyin çağrışımları göz ardı edilerek ham hali ele alınışı anlamsızdır",
    "iki farklı rüya aynı tipik temayı sergiliyorken içeriğin anlatısı tamamen farklı olabilir",
]


for idx, cümle in enumerate(örnek_cümleler, 1):
    benzer_cümleler = benzer_cumleleri_bul(model, cümle)
    print(f"{idx}. cümleye benzer cümleler:")
    for benzer_cümle in benzer_cümleler:
        print("-", benzer_cümle)
    print()


1. cümleye benzer cümleler:
- 28025
- 7
- 12349
- 4445
- 18136
- 14647

2. cümleye benzer cümleler:
- 26918
- 12349
- 18136
- 14673
- 15063
- 29629

3. cümleye benzer cümleler:
- 28091
- 44
- 28104
- 2348
- 2412
- 30558

4. cümleye benzer cümleler:
- 29331
- 25102
- 28098
- 25471
- 31413
- 2615

5. cümleye benzer cümleler:
- 49
- 29638
- 1006
- 851
- 32044
- 28108



C:\Users\muham\AppData\Local\Temp\ipykernel_33564\2112184786.py:15: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  benzer_cümleler = model.docvecs.most_similar([cümle_vektoru], topn=n)
